In [65]:
import numpy as np
import pandas as pd


def truncate_float_str(s: str | float) -> str:
    # check if is nan
    if isinstance(s, float):
        return ""
    return s.split(",")[0]


points_df = pd.read_csv("~/Downloads/points.csv")
points_df = points_df[["Student", "Контрольная работа (55637923)"]]
points_df = points_df.rename(columns={"Student": "name_short", "Контрольная работа (55637923)": "points"})
points_df["points"] = points_df["points"].apply(truncate_float_str)

all_names_df = pd.read_excel("~/Downloads/all_names.xlsx")
all_names_df["name_short"] = all_names_df.name.apply(lambda s: s.split()[0].lower())
dupl_names: list[str] = all_names_df.name_short[all_names_df.name_short.duplicated()].to_list()
all_names_df

,name,group,name_short
0,Абгарян Маргарита Овакимовна,ББИ2401,абгарян
1,Андреев Артём Андреевич,ББИ2401,андреев
2,Афанасиева Арина Руслановна,ББИ2401,афанасиева
3,Болокан Дмитрий Александрович,ББИ2401,болокан
4,Валиуллин Демид Раисович,ББИ2401,валиуллин
...,...,...,...
326,Цю-жен-цин Анита Андреевна,ББИ2411,цю-жен-цин
327,Чепурной Дмитрий Евгеньевич,ББИ2411,чепурной
328,Чистяков Никита Алексеевич,ББИ2411,чистяков
329,Чудайкин Игорь Александрович,ББИ2411,чудайкин


In [66]:
dupl_names

['смолина', 'анисимова', 'мартынов', 'васильев', 'зуева', 'колесников']

In [67]:
from functools import partial


def rename(s: str, dupl_names: list[str] | None = None) -> str:
    s_split = s.lower().split()
    res: str = ""
    if len(s_split) == 2 or len(s_split) == 3 and s_split[0][-1] == ",":
        res = s_split[1]
    elif len(s_split) == 3:
        res = s_split[0]
    elif len(s_split) == 4 and s_split[0][-1] == ",":
        res = s_split[1]

    if res in (dupl_names or []):
        return ""
    return res


rename_func_no_duplicate = partial(rename, dupl_names=dupl_names)
points_df["name_short"] = points_df["name_short"].apply(rename_func_no_duplicate)

In [ ]:
res_df = pd.merge(all_names_df, points_df, on="name_short", how="left")
res_df = res_df.drop_duplicates(subset=["name"])
res_df.to_excel("~/Downloads/output.xlsx")
res_df

,name,group,name_short,points
0,Абгарян Маргарита Овакимовна,ББИ2401,абгарян,22
1,Андреев Артём Андреевич,ББИ2401,андреев,NaN
2,Афанасиева Арина Руслановна,ББИ2401,афанасиева,22
3,Болокан Дмитрий Александрович,ББИ2401,болокан,NaN
4,Валиуллин Демид Раисович,ББИ2401,валиуллин,NaN
...,...,...,...,...
334,Цю-жен-цин Анита Андреевна,ББИ2411,цю-жен-цин,23
335,Чепурной Дмитрий Евгеньевич,ББИ2411,чепурной,24
336,Чистяков Никита Алексеевич,ББИ2411,чистяков,NaN
337,Чудайкин Игорь Александрович,ББИ2411,чудайкин,19
